In [29]:
# imports and setup 
from bs4 import BeautifulSoup
# you can use either of these libraries to get html from a website
import requests
import urllib.request
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re

import pandas as pd
import scipy as sc
import numpy as np
import time
import statsmodels.formula.api as sm

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline 
plt.rcParams['figure.figsize'] = (10, 6) 
from string import ascii_lowercase

In [3]:
url = "https://www.basketball-reference.com/players"
all_letters = ""
html = {}
i = 0
for x in ascii_lowercase:
    if x == 'x':
        pass
    else:
        link = url + '/' + str(x) + '/'
        with urllib.request.urlopen(link) as response:
            html[i] = response.read()
            html[i] = html[i].decode('utf-8')
        all_letters = all_letters + str(html[i])
        i += 1

In [4]:
##Don't run again

with open('all_letters.html','w') as new_file:
    new_file.write(all_letters)

In [5]:
letters_soup = BeautifulSoup(open('all_letters.html',encoding = 'utf-8'), "html.parser")

In [6]:
links = []
url = "https://www.basketball-reference.com"
for x in letters_soup.find_all(class_="left "):
    if x.get("data-stat") == "player":
        link_tail = x.find("a").get("href")
        link = url + str(link_tail)
        links.append(link)


In [106]:
all_players = ""
for i in range(0, len(links)):
    url = links[i]
    html = {}
    with urllib.request.urlopen(url) as response:
        html[i] = response.read()
        html[i] = html[i].decode('utf-8')
    all_players = all_players + str(html[i])
    time.sleep(3)
with open('all_players.html','w') as new_file:
    new_file.write(all_players)


In [7]:
len(links)

4569

In [5]:
players_soup = BeautifulSoup(open('all_players.html',encoding = 'utf-8'), "html.parser")

KeyboardInterrupt: 

In [1]:
print(players_soup.prettify()[:500000])

NameError: name 'players_soup' is not defined

In [133]:
rookie = []
columns = ['Name','Year','Age','Team','League','Position','Games Played','Games Started','MinPerGame','FGPerGame','FGAPerGame','FG%PerGame','3pPerGame','3PAPerGame','3p%PerGame','2pPerGame','2pAPerGame','2p%PerGame','eFG%PerGame','FTPerGame','FTAPerGame','FT%PerGame','ORGPerGame','DRBPerGame','TotalRebPerGame','AstPerGame','StealPerGame','BlkPerGame','TOVPerGame','FoulsPerGame','PtsPerGame']
df_pergame = pd.DataFrame(columns = columns)
names_not_included = []
for i in range(0, 4):
    url = links[i]
    html = {}
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')
    players_soup = BeautifulSoup(html,'html.parser')
    time.sleep(3)
    for x in players_soup.find_all(id="per_game"):
        for q in players_soup.find("h1"):
            rookie.append(q)
        for y in x.find_all("tr")[1]:
            rookie.append(y.get_text())
        if len(rookie) == 31:
            df_pergame.loc[i] = rookie
        else:
            for p in players_soup.find("h1"):
                names_not_included.append(p)
        rookie = []

In [93]:
df_pergame.to_csv("per_game.csv")

In [21]:
start_time = time.time()
rookie = []
columns_per36 = ['Name','Year','Age','Team','League','Position','Games Played','Games Started','TotalMin','FGPer36','FGAPer36','FG%Per36','3pPer36','3PAPer36','3p%Per36','2pPer36','2pAPer36','2p%Per36','FTPer36','FTAPer36','FT%Per36','ORGPer36','DRBPer36','TotalRebPer36','AstPer36','StealPer36','BlkPer36','TOVPer36','FoulsPer36','PtsPer36']
df_per36 = pd.DataFrame(columns = columns_per36)
columns_per100 = ['Name','Year','Age','Team','League','Position','Games Played','Games Started','TotalMin','FGPer100Poss','FGAPer100Poss','FG%Per100Poss','3pPer100Poss','3PAPer100Poss','3p%Per100Poss','2pPer100Poss','2pAPer100Poss','2p%Per100Poss','FTPer100Poss','FTAPer100Poss','FT%Per100Poss','ORGPer100Poss','DRBPer100Poss','TotalRebPer100Poss','AstPer100Poss','StealPer100Poss','BlkPer100Poss','TOVPer100Poss','FoulsPer100Poss','PtsPer100Poss','blank','ORtg','DRtg']
df_per100 = pd.DataFrame(columns = columns_per100)
columns_advanced = ['Name','Year','Age','Team','League','Position','Games Played','TotalMin','PER','TS%','3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','blank','OWS','DWS','WS','WS/48','blank','OBPM','DBPM','BPM','VORP']
df_advanced = pd.DataFrame(columns = columns_advanced)
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(chrome_options=chrome_options)
for i in range(0, len(links)):
    url = links[i]
    driver.get(url)
    players_soup = BeautifulSoup(driver.page_source,'html.parser')
    for x in players_soup.find_all(id="per_minute"):
        for q in players_soup.find("h1"):
            rookie.append(q)
        for y in x.find_all("tr")[1]:
            rookie.append(y.get_text())
        if len(rookie) == 30:
            df_per36.loc[i] = rookie
        rookie = []
    for x in players_soup.find_all(id="per_poss"):
        for q in players_soup.find("h1"):
            rookie.append(q)
        for y in x.find_all("tr")[1]:
            rookie.append(y.get_text())
        if len(rookie) == 33:
            df_per100.loc[i] = rookie
        rookie = []  
    for x in players_soup.find_all(id="advanced"):
        for q in players_soup.find("h1"):
            rookie.append(q)
        for y in x.find_all("tr")[1]:
            rookie.append(y.get_text())
        if len(rookie) == 30:
            df_advanced.loc[i] = rookie
        rookie = []
driver.close()

In [124]:
df_advanced.to_csv('rookie_advanced.csv')

In [125]:
df_per100.to_csv('per_100.csv')

In [22]:
df_per36.to_csv('per_36.csv')

In [ ]:
rookie = []
columns = ['Name','Year','Age','Team','League','Position','Games Played','Games Started','TotalMin','FGPer100Poss','FGAPer100Poss','FG%Per100Poss','3pPer100Poss','3PAPer100Poss','3p%Per100Poss','2pPer100Poss','2pAPer100Poss','2p%Per100Poss','FTPer100Poss','FTAPer100Poss','FT%Per100Poss','ORGPer100Poss','DRBPer100Poss','TotalRebPer100Poss','AstPer100Poss','StealPer100Poss','BlkPer100Poss','TOVPer100Poss','FoulsPer100Poss','PtsPer100Poss']
df_per100 = pd.DataFrame(columns = columns)
names_not_included = []
for i in range(0, 5):
    url = links[i]
    driver = webdriver.Chrome()
    driver.get(url)
    players_soup = BeautifulSoup(driver.page_source,'html.parser')
    time.sleep(3)
    for x in players_soup.find_all(id="per_poss"):
        for q in players_soup.find("h1"):
            rookie.append(q)
        for y in x.find_all("tr")[1]:
            rookie.append(y.get_text())
        if len(rookie) == 30:
            df_per100.loc[i] = rookie
        rookie = []
    driver.close()

In [52]:
names_not_included

['Zaid Abdul-Aziz',
 'Forest Able',
 'John Abramovic',
 'Don Ackerman',
 'Bud Acton']

In [144]:
career = []
columns = ['Name','Year','blank','blank','League','blank','Games Played','Games Started','MinPerGame','FGPerGame','FGAPerGame','FG%PerGame','3pPerGame','3PAPerGame','3p%PerGame','2pPerGame','2pAPerGame','2p%PerGame','eFG%PerGame','FTPerGame','FTAPerGame','FT%PerGame','ORGPerGame','DRBPerGame','TotalRebPerGame','AstPerGame','StealPerGame','BlkPerGame','TOVPerGame','FoulsPerGame','PtsPerGame']
career_pergame = pd.DataFrame(columns = columns)
names_not_included = []
for i in range(0, len(links)):
    url = links[i]
    html = {}
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')
    players_soup = BeautifulSoup(html,'html.parser')
    for x in players_soup.find_all(id="per_game"):
        for q in players_soup.find("h1"):
            career.append(q)
        for y in x.find_all("tfoot"):
            for z in y.find_all("tr")[0]:
                career.append(z.get_text())
        if len(career) == 31:
            career_pergame.loc[i] = career
        else:
            for p in players_soup.find("h1"):
                names_not_included.append(p)
        career = []

In [155]:
career_pergame.to_csv('career_pergame.csv')

In [10]:
start_time = time.time()
career = []
columns_per36 = ['Name','Year','blank','blank','League','blank','Games Played','Games Started','TotalMin','FGPer36','FGAPer36','FG%Per36','3pPer36','3PAPer36','3p%Per36','2pPer36','2pAPer36','2p%Per36','FTPer36','FTAPer36','FT%Per36','ORGPer36','DRBPer36','TotalRebPer36','AstPer36','StealPer36','BlkPer36','TOVPer36','FoulsPer36','PtsPer36']
career_per36 = pd.DataFrame(columns = columns_per36)
columns_per100 = ['Name','Year','blank','blank','League','blank','Games Played','Games Started','TotalMin','FGPer100Poss','FGAPer100Poss','FG%Per100Poss','3pPer100Poss','3PAPer100Poss','3p%Per100Poss','2pPer100Poss','2pAPer100Poss','2p%Per100Poss','FTPer100Poss','FTAPer100Poss','FT%Per100Poss','ORGPer100Poss','DRBPer100Poss','TotalRebPer100Poss','AstPer100Poss','StealPer100Poss','BlkPer100Poss','TOVPer100Poss','FoulsPer100Poss','PtsPer100Poss','blank','ORtg','DRtg']
career_per100 = pd.DataFrame(columns = columns_per100)
columns_advanced = ['Name','Year','blank','blank','League','blank','Games Played','TotalMin','PER','TS%','3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','blank','OWS','DWS','WS','WS/48','blank','OBPM','DBPM','BPM','VORP']
career_advanced = pd.DataFrame(columns = columns_advanced)
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(chrome_options=chrome_options)
for i in range(0, len(links)):
    url = links[i]
    driver.get(url)
    players_soup = BeautifulSoup(driver.page_source,'html.parser')
    for x in players_soup.find_all(id="per_minute"):
        for q in players_soup.find("h1"):
            career.append(q)
        for z in x.find_all("tfoot"):
            for y in z.find_all("tr")[0]:
                career.append(y.get_text())
        if len(career) == 30:
            career_per36.loc[i] = career
        career = []
    for x in players_soup.find_all(id="per_poss"):
        for q in players_soup.find("h1"):
            career.append(q)
        for z in x.find_all("tfoot"):
            for y in z.find_all("tr")[0]:
                career.append(y.get_text())
        if len(career) == 33:
            career_per100.loc[i] = career
        career = []  
    for x in players_soup.find_all(id="advanced"):
        for q in players_soup.find("h1"):
            career.append(q)
        for z in x.find_all("tfoot"):
            for y in z.find_all("tr")[0]:
                career.append(y.get_text())
        if len(career) == 30:
            career_advanced.loc[i] = career
        career = []
    print(str(i) + ": " + str(time.time()-start_time))
driver.close()

0: 9.259314060211182
1: 12.186713218688965
2: 16.080271244049072
3: 18.133547067642212
4: 20.184301137924194
5: 22.500582218170166
6: 24.445981979370117
7: 25.50577688217163
8: 26.570744037628174
9: 28.38466715812683
10: 29.86523127555847
11: 30.997079133987427
12: 32.80538010597229
13: 33.75652003288269
14: 35.610326051712036
15: 38.17450022697449
16: 40.59020709991455
17: 42.09175395965576
18: 43.74794292449951
19: 45.33284306526184
20: 47.68596410751343
21: 49.58579611778259
22: 51.54961705207825
23: 53.067835330963135
24: 54.80564093589783
25: 56.74911904335022
26: 59.43418025970459
27: 61.3753981590271
28: 63.68875813484192
29: 65.78358197212219
30: 69.52075409889221
31: 71.23128509521484
32: 73.34308409690857
33: 74.78493809700012
34: 76.08801412582397
35: 77.51868009567261
36: 79.46544313430786
37: 83.96216106414795
38: 85.25000309944153
39: 87.6593370437622
40: 90.30405616760254
41: 91.55490612983704
42: 92.82693004608154
43: 95.06074213981628
44: 97.15249800682068
45: 98.39985

357: 649.0173621177673
358: 650.1679401397705
359: 651.3529872894287
360: 652.6663970947266
361: 654.2136151790619
362: 657.0585200786591
363: 658.3974480628967
364: 660.3714301586151
365: 662.6834859848022
366: 665.1905422210693
367: 666.8082931041718
368: 669.0268790721893
369: 670.1146221160889
370: 671.849766254425
371: 672.7338771820068
372: 673.9376382827759
373: 675.1219341754913
374: 676.7845830917358
375: 677.8363411426544
376: 679.2721259593964
377: 681.2452852725983
378: 683.8907992839813
379: 684.9480550289154
380: 686.2574911117554
381: 687.5700590610504
382: 689.4633610248566
383: 695.7464530467987
384: 698.4149188995361
385: 701.0912511348724
386: 702.5325729846954
387: 704.0709562301636
388: 706.6886551380157
389: 708.1166040897369
390: 709.2576591968536
391: 710.6614661216736
392: 711.6626281738281
393: 712.8843379020691
394: 715.7115750312805
395: 716.9953620433807
396: 718.1473381519318
397: 719.9906129837036
398: 721.3208100795746
399: 722.5746850967407
400: 724.073

710: 1291.232143163681
711: 1293.4499552249908
712: 1295.6558201313019
713: 1296.9375569820404
714: 1298.1489169597626
715: 1299.8295249938965
716: 1301.2016382217407
717: 1302.824788093567
718: 1304.0471801757812
719: 1305.3431622982025
720: 1307.7992541790009
721: 1309.7219111919403
722: 1311.6323132514954
723: 1313.2054171562195
724: 1314.3979060649872
725: 1315.7111320495605
726: 1316.9948332309723
727: 1318.1508140563965
728: 1319.3488252162933
729: 1320.390753030777
730: 1321.317509174347
731: 1322.5098760128021
732: 1323.765673160553
733: 1325.1637411117554
734: 1327.3544383049011
735: 1329.0053172111511
736: 1331.6563940048218
737: 1333.0643799304962
738: 1334.5810949802399
739: 1335.6637651920319
740: 1338.080197095871
741: 1339.7351932525635
742: 1340.9634470939636
743: 1343.8824620246887
744: 1346.0037231445312
745: 1347.2743110656738
746: 1349.9214899539948
747: 1352.990098953247
748: 1354.31458902359
749: 1355.6212120056152
750: 1357.2693090438843
751: 1358.6733450889587
7

1053: 1939.8674120903015
1054: 1941.943192243576
1055: 1944.1463890075684
1056: 1946.6991860866547
1057: 1948.0302891731262
1058: 1950.8307530879974
1059: 1952.0426099300385
1060: 1954.1284301280975
1061: 1956.1506991386414
1062: 1957.416020154953
1063: 1959.1839849948883
1064: 1960.0957281589508
1065: 1962.608694076538
1066: 1964.1229960918427
1067: 1966.865313053131
1068: 1969.4626071453094
1069: 1971.1646132469177
1070: 1972.3986361026764
1071: 1973.9135320186615
1072: 1976.6098160743713
1073: 1978.008219242096
1074: 1979.390174150467
1075: 2049.089518070221
1076: 2050.40865111351
1077: 2052.064966201782
1078: 2053.5390799045563
1079: 2054.900698184967
1080: 2057.9765939712524
1081: 2060.587213039398
1082: 2064.0639040470123
1083: 2065.7407319545746
1084: 2066.7290031909943
1085: 2068.8447000980377
1086: 2071.1311061382294
1087: 2074.181133031845
1088: 2075.7495329380035
1089: 2076.807267189026
1090: 2078.021175146103
1091: 2079.2716591358185
1092: 2080.4695591926575
1093: 2081.7507

1388: 2662.9543759822845
1389: 2665.700859308243
1390: 2666.8114931583405
1391: 2668.0792560577393
1392: 2669.5976650714874
1393: 2671.1310040950775
1394: 2673.4944450855255
1395: 2675.7390551567078
1396: 2677.220659971237
1397: 2679.620271205902
1398: 2681.1751539707184
1399: 2683.517314195633
1400: 2685.364729166031
1401: 2687.5166521072388
1402: 2690.161222219467
1403: 2691.9630811214447
1404: 2695.573940038681
1405: 2696.7718200683594
1406: 2699.0175511837006
1407: 2700.836935043335
1408: 2703.195081949234
1409: 2704.3229172229767
1410: 2705.30899810791
1411: 2706.5493891239166
1412: 2707.5891630649567
1413: 2709.2743213176727
1414: 2712.222419023514
1415: 2713.730724096298
1416: 2714.9209821224213
1417: 2717.285100221634
1418: 2720.0083820819855
1419: 2721.2044649124146
1420: 2722.182890176773
1421: 2723.2680971622467
1422: 2725.8290622234344
1423: 2727.616715192795
1424: 2730.3435909748077
1425: 2731.722938299179
1426: 2733.0531389713287
1427: 2736.2694461345673
1428: 2739.153439

...

3085: 6648.599023103714
3086: 6650.028267145157
3087: 6651.003926038742
3088: 6652.069070100784
3089: 6653.16832613945
3090: 6654.620462179184
3091: 6655.8265681266785
3092: 6658.610419988632
3093: 6660.026762008667
3094: 6661.829870223999
3095: 6663.294628143311
3096: 6665.935641288757
3097: 6668.648348093033
3098: 6669.918063163757
3099: 6671.800859212875
3100: 6674.27813410759
3101: 6675.406337022781
3102: 6676.763570070267
3103: 6677.747684240341
3104: 6679.671641111374
3105: 6681.107215166092
3106: 6682.334815979004
3107: 6683.623384237289
3108: 6684.784709215164
3109: 6686.314289093018
3110: 6687.660119056702
3111: 6689.043078184128
3112: 6690.777350187302
3113: 6694.134056091309
3114: 6695.832585096359
3115: 6697.107413053513
3116: 6699.8666570186615
3117: 6701.8501262664795
3118: 6704.138052225113
3119: 6705.702704906464
3120: 6708.041314125061
3121: 6709.678115129471
3122: 6712.556784152985
3123: 6714.218041181564
3124: 6715.115951299667
3125: 6716.2239382267
3126: 6718.655744

3426: 7229.8747770786285
3427: 7231.862342119217
3428: 7234.496175289154
3429: 7236.330676078796
3430: 7237.71609711647
3431: 7239.8638920784
3432: 7241.938155174255
3433: 7244.156081914902
3434: 7246.077548027039
3435: 7247.413082122803
3436: 7248.854328155518
3437: 7250.95566534996
3438: 7253.097722291946
3439: 7254.551707267761
3440: 7257.064050912857
3441: 7260.549453020096
3442: 7263.188894987106
3443: 7265.1306891441345
3444: 7267.325951099396
3445: 7269.73083615303
3446: 7271.938454151154
3447: 7273.416934251785
3448: 7274.737622022629
3449: 7276.829297065735
3450: 7278.762976169586
3451: 7281.339869022369
3452: 7282.782963037491
3453: 7283.992655277252
3454: 7286.081678152084
3455: 7287.296910047531
3456: 7289.440560102463
3457: 7291.78985118866
3458: 7293.095063209534
3459: 7294.349043130875
3460: 7296.171449184418
3461: 7298.900896072388
3462: 7300.456548213959
3463: 7302.661292076111
3464: 7303.875202178955
3465: 7304.784765005112
3466: 7306.456248044968
3467: 7309.258109092

3769: 8281.286208152771
3770: 8283.905012130737
3771: 8286.360401153564
3772: 8289.38093495369
3773: 8296.527140140533
3774: 8298.79680109024
3775: 8305.738381147385
3776: 8308.164828062057
3777: 8309.819462299347
3778: 8312.677603244781
3779: 8314.535908222198
3780: 8317.074356079102
3781: 8318.910846948624
3782: 8320.81318807602
3783: 8323.305195093155
3784: 8324.672973155975
3785: 8326.104463100433
3786: 8328.190489053726
3787: 8329.476114034653
3788: 8331.540379285812
3789: 8333.979215145111
3790: 8335.312516212463
3791: 8337.671123027802
3792: 8339.514125108719
3793: 8341.166363954544
3794: 8342.556256055832
3795: 8346.066964149475
3796: 8347.97861623764
3797: 8349.164844036102
3798: 8350.53882408142
3799: 8353.52759718895
3800: 8354.775270938873
3801: 8356.430918216705
3802: 8358.60537815094
3803: 8361.118396043777
3804: 8362.129774093628
3805: 8364.516000032425
3806: 8367.79271197319
3807: 8370.004836082458
3808: 8372.140400886536
3809: 8373.47273516655
3810: 8375.891096115112
3

4114: 8955.725672960281
4115: 8957.104141235352
4116: 8959.639839172363
4117: 8962.127388954163
4118: 8964.480976104736
4119: 8970.746174097061
4120: 8972.64967918396
4121: 8976.897497177124
4122: 8979.13576412201
4123: 8980.07969903946
4124: 8981.933032989502
4125: 8986.945835113525
4126: 8988.87025809288
4127: 8990.443957090378
4128: 8992.49455690384
4129: 8996.218668222427
4130: 8997.930026054382
4131: 8999.309647083282
4132: 9000.26773929596
4133: 9002.007258176804
4134: 9003.718055963516
4135: 9004.999866247177
4136: 9007.275696992874
4137: 9010.195984125137
4138: 9013.199600934982
4139: 9015.018752098083
4140: 9016.232726097107
4141: 9017.602542161942
4142: 9018.817615270615
4143: 9020.103979110718
4144: 9022.739253997803
4145: 9024.259314060211
4146: 9026.147208929062
4147: 9028.201800107956
4148: 9029.884239196777
4149: 9032.922723054886
4150: 9035.163194179535
4151: 9037.994312047958
4152: 9040.114520072937
4153: 9041.53612112999
4154: 9043.67724609375
4155: 9046.992407083511


4455: 10193.561205148697
4456: 10194.72872018814
4457: 10195.626793146133
4458: 10197.23528122902
4459: 10198.38368010521
4460: 10199.56934928894
4461: 10200.860594034195
4462: 10202.35120010376
4463: 10203.314959287643
4464: 10204.892033100128
4465: 10206.351793050766
4466: 10207.970894098282
4467: 10209.138112068176
4468: 10210.118767261505
4469: 10211.564726114273
4470: 10214.172894954681
4471: 10216.071634054184
4472: 10217.425466299057
4473: 10218.908292293549
4474: 10220.620390892029
4475: 10221.739378929138
4476: 10222.865669250488
4477: 10225.12944316864
4478: 10226.534342288971
4479: 10227.533000946045
4480: 10229.764262199402
4481: 10231.716497898102
4482: 10232.942820072174
4483: 10234.189252138138
4484: 10236.190640211105
4485: 10237.57415819168
4486: 10239.290143013
4487: 10241.688902139664
4488: 10243.116260290146
4489: 10244.645520210266
4490: 10246.418872117996
4491: 10247.309375047684
4492: 10248.740097045898
4493: 10250.761349201202
4494: 10251.924290180206
4495: 1025

In [18]:
career_per36.to_csv('career_per36.csv')

In [19]:
career_per100.to_csv('career_per100.csv')

In [20]:
career_advanced.to_csv('career_advanced.csv')

In [ ]:
rookie_pergame=pd.read_csv('rookie_per')

In [35]:
def get_num(x):
    return int(''.join(ele for ele in x if ele.isdigit()))

In [42]:
#Scrape career attributes like MVP's, All-Star Games, All-NBA Teams, All-Rookie Team, ROY
awards = []
columns = ['Name','MVPs','All-Star Games','All-NBA Teams','All-Rookie Team','ROY']
career_awards = pd.DataFrame(columns = columns)
zeros = [0,0,0,0,0,0]
for i in range(0, len(links)):
    url = links[i]
    html = {}
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')
    players_soup = BeautifulSoup(html,'html.parser')
    for x in players_soup.find_all(id="info"):
        if x.find_all(id="bling") != []:
            career_awards.loc[i] = zeros
            for y in x.find_all("li"):
                for q in players_soup.find("h1"):
                    career_awards['Name'].loc[i] = q
                result = y.get_text()
                if "x MVP" in result:
                    career_awards['MVPs'].loc[i] = get_num(result)
                if "All Star" in result:
                    career_awards['All-Star Games'].loc[i] = get_num(result)
                if "All-NBA" in result:
                    if get_num(result) > 20:
                        career_awards['All-NBA Teams'].loc[i] = 1
                    else:
                        career_awards['All-NBA Teams'].loc[i] = get_num(result)
                if "All-Rookie" in result:
                    career_awards['All-Rookie Team'].loc[i] = 1
                if "ROY" in result:
                    career_awards['ROY'].loc[i] = 1
                result = ""
        else:
            career_awards.loc[i] = zeros
            for q in players_soup.find("h1"):
                career_awards['Name'].loc[i] = q
    print(i)
                    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


In [44]:
career_awards.to_csv('career_awards.csv')

In [21]:
for i in range(0, 4):
    url = links[i]
    html = {}
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')
    players_soup = BeautifulSoup(html,'html.parser')
    for x in players_soup.find_all(id="info"):
        if x.find_all(id="bling") != []:
            for y in x.find_all("li"):
                print(y.get_text())
                print("----")
            print(123)

Hall of Fame
----
19x All Star
----
2x Scoring Champ
----
1975-76 TRB Champ
----
4x BLK Champ
----
6x NBA Champ
----
15x All-NBA
----
11x All-Defensive
----
1969-70 All-Rookie
----
2x Finals MVP
----
6x MVP
----
1969-70 ROY
----
123
1990-91 All-Rookie
----
1992-93 Most Improved
----
123
